In [1432]:
import pandas as pd
import numpy as np
import math
from scipy import stats
from sklearn import preprocessing
import seaborn as sns
import matplotlib.pyplot as plt
from functools import reduce
from sklearn import preprocessing
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from imblearn.ensemble import BalancedBaggingClassifier
from xgboost import XGBClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier


from imblearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from imblearn.pipeline import make_pipeline as imbalanced_make_pipeline
from imblearn.over_sampling import SMOTE
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, confusion_matrix, classification_report
import warnings
warnings.filterwarnings("ignore")

### Reading required Train & Test Data without Duplicates

In [1355]:
acc_train = pd.read_csv("raw_account_70_new.csv")
data_train = pd.read_csv("raw_data_70_new.csv")
enq_train = pd.read_csv("raw_enquiry_70_new.csv")
acc_test = pd.read_csv("raw_account_30_new.csv")
data_test = pd.read_csv("raw_data_30_new.csv")
enq_test = pd.read_csv("raw_enquiry_30_new.csv")
acc_train.drop_duplicates(inplace = True)
acc_test.drop_duplicates(inplace = True)
enq_train.drop_duplicates(inplace = True)
enq_test.drop_duplicates(inplace = True)
data_train.drop_duplicates(inplace = True)
data_test.drop_duplicates(inplace = True)

### Account Train data pre-processing

After exploring all the features in the Datasets, I have found best to create new features from the available data which could be useful based on intuition and also created 10 of the suggested features for the data. Columns containing irrelavant information, uncertain information and correlated coulumns have been dropped.

In [1332]:
#secured account values
sec_ln = [1.0,2.0,3.0,4.0,7.0,11.0,13.0,14.0,15.0,17.0,31.0,32.0,33.0,34.0,42.0,51.0,52.0,53.0,54.0,55.0,56.0,57.0,58.0,
          59.0,60.0]

#value to use for null values in opened_dt by adding it to payment_end_dt and impute it
imp1 = (pd.to_datetime(acc_train["opened_dt"])-pd.to_datetime(acc_train["paymt_end_dt"])).dt.days
imp1 = imp1.mean()
imp1


acc_train['mean_accs_missing_last_paymt_dt'] = np.where(acc_train['last_paymt_dt'].isnull(),1,0)
grp1=pd.DataFrame(acc_train.groupby(["customer_no"])["mean_accs_missing_last_paymt_dt"].mean())
grp1 = grp1.reset_index(level='customer_no')
grp2=pd.DataFrame(acc_train.groupby(["customer_no"])["mean_accs_missing_last_paymt_dt"].sum())
grp2.rename(columns={'mean_accs_missing_last_paymt_dt': 'total_accs_missing_last_paymt_dt'}, inplace=True)
grp2 = grp2.reset_index(level='customer_no')

# fill NA values in last payment date with reporting date values.
# create feature null or not last payment date
acc_train['last_paymt_dt'] = np.where(acc_train['last_paymt_dt'].isnull(), acc_train['reporting_dt'], acc_train['last_paymt_dt'] )
acc_train['mean_diff_lastpaymt_opened_dt'] = (pd.to_datetime(acc_train["last_paymt_dt"])-pd.to_datetime(acc_train["opened_dt"])).dt.days
acc_train['mean_diff_lastpaymt_opened_dt'] = np.where(acc_train['mean_diff_lastpaymt_opened_dt'].isnull(), (pd.to_datetime(acc_train["last_paymt_dt"])-pd.to_datetime(acc_train["paymt_end_dt"])).dt.days - imp1, acc_train['mean_diff_lastpaymt_opened_dt'] )
grp3=pd.DataFrame(acc_train.groupby(["customer_no"])["mean_diff_lastpaymt_opened_dt"].mean())
grp3 = grp3.reset_index(level='customer_no')
grp4=pd.DataFrame(acc_train.groupby(["customer_no"])["mean_diff_lastpaymt_opened_dt"].sum())
grp4.rename(columns={'mean_diff_lastpaymt_opened_dt': 'total_diff_lastpaymt_opened_dt'}, inplace=True)
grp4 = grp4.reset_index(level='customer_no')

acc_train['payment_history_mean_length'] = ((pd.to_datetime(acc_train["paymt_str_dt"])-pd.to_datetime(acc_train["paymt_end_dt"]))/np.timedelta64(1, 'M'))
acc_train['payment_history_mean_length'] = np.where(acc_train['payment_history_mean_length'].isnull(), acc_train['payment_history_mean_length'].mean(), acc_train['payment_history_mean_length'] )
acc_train['payment_history_mean_length'] = acc_train['payment_history_mean_length'].astype(int)
grp5=pd.DataFrame(acc_train.groupby(["customer_no"])["payment_history_mean_length"].mean())
grp5 = grp5.reset_index(level='customer_no')
grp6=pd.DataFrame(acc_train.groupby(["customer_no"])["payment_history_mean_length"].sum())
grp6.rename(columns={'payment_history_mean_length': 'payment_history_total_length'}, inplace=True)
grp6 = grp6.reset_index(level='customer_no')

acc_train['ratio_closed_accs'] = np.where(acc_train['closed_dt'].isnull(),1,0)
grp7=pd.DataFrame(acc_train.groupby(["customer_no"])["ratio_closed_accs"].mean())
grp7 = grp7.reset_index(level='customer_no')
grp8=pd.DataFrame(acc_train.groupby(["customer_no"])["ratio_closed_accs"].sum())
grp8.rename(columns={'ratio_closed_accs': 'total_closed_accs'}, inplace=True)
grp8 = grp8.reset_index(level='customer_no')

acc_train['most_common_account'] = acc_train['acct_type']
grp9 = pd.DataFrame(acc_train.groupby(['customer_no'])['most_common_account'].apply(lambda x: x.mode()))
grp9 = grp9.xs(0, level=1)
grp9 = grp9.reset_index(level='customer_no')

acc_train['secured_accounts_ratio'] = np.where(acc_train['acct_type'].isin(sec_ln),1,0)
grp10=pd.DataFrame(acc_train.groupby(["customer_no"])["secured_accounts_ratio"].mean())
grp10 = grp10.reset_index(level='customer_no')
grp11=pd.DataFrame(acc_train.groupby(["customer_no"])["secured_accounts_ratio"].sum())
grp11.rename(columns={'secured_accounts_ratio': 'secured_accounts_total'}, inplace=True)
grp11 = grp11.reset_index(level='customer_no')

acc_train['creditcard_accounts_ratio'] = np.where(acc_train['acct_type'] == 10.0,1,0)
grp12=pd.DataFrame(acc_train.groupby(["customer_no"])["creditcard_accounts_ratio"].mean())
grp12 = grp12.reset_index(level='customer_no')
grp13=pd.DataFrame(acc_train.groupby(["customer_no"])["creditcard_accounts_ratio"].sum())
grp13.rename(columns={'creditcard_accounts_ratio': 'total_creditcard_accounts'}, inplace=True)
grp13 = grp13.reset_index(level='customer_no')

acc_train['mean_accs_individual_owner'] = np.where(acc_train['owner_indic'] == 1,1,0)
grp14=pd.DataFrame(acc_train.groupby(["customer_no"])["mean_accs_individual_owner"].mean())
grp14 = grp14.reset_index(level='customer_no')
grp15=pd.DataFrame(acc_train.groupby(["customer_no"])["mean_accs_individual_owner"].sum())
grp15.rename(columns={'mean_accs_individual_owner': 'total_accs_individual_owner'}, inplace=True)
grp15 = grp15.reset_index(level='customer_no')

acc_train['mean_accs_with_amt_past_due'] = np.where(acc_train['amt_past_due'].isnull(),0,1)
grp16=pd.DataFrame(acc_train.groupby(["customer_no"])["mean_accs_with_amt_past_due"].mean())
grp16 = grp16.reset_index(level='customer_no')
grp17=pd.DataFrame(acc_train.groupby(["customer_no"])["mean_accs_with_amt_past_due"].sum())
grp17.rename(columns={'mean_accs_with_amt_past_due': 'total_accs_with_amt_past_due'}, inplace=True)
grp17 = grp17.reset_index(level='customer_no')

acc_train['mean_amt_past_due'] = acc_train['amt_past_due']
acc_train['mean_amt_past_due'] = np.where(acc_train['mean_amt_past_due'].isnull(), 0, acc_train['mean_amt_past_due'] )
grp18=pd.DataFrame(acc_train.groupby(["customer_no"])["mean_amt_past_due"].mean())
grp18 = grp18.reset_index(level='customer_no')
grp19=pd.DataFrame(acc_train.groupby(["customer_no"])["mean_amt_past_due"].sum())
grp19.rename(columns={'mean_amt_past_due': 'total_amt_past_due'}, inplace=True)
grp19 = grp19.reset_index(level='customer_no')

acc_train['mean_accs_missing_paymentfrequency'] = np.where(acc_train['paymentfrequency'].isnull(),1,0)
grp20=pd.DataFrame(acc_train.groupby(["customer_no"])["mean_accs_missing_paymentfrequency"].mean())
grp20 = grp20.reset_index(level='customer_no')
grp21=pd.DataFrame(acc_train.groupby(["customer_no"])["mean_accs_missing_paymentfrequency"].sum())
grp21.rename(columns={'mean_accs_missing_paymentfrequency': 'total_accs_missing_paymentfrequency'}, inplace=True)
grp21 = grp21.reset_index(level='customer_no')

acc_train['mean_accs_weekly_paymentfrequency'] = np.where(acc_train['paymentfrequency'] == 1,1,0)
grp22=pd.DataFrame(acc_train.groupby(["customer_no"])["mean_accs_weekly_paymentfrequency"].mean())
grp22 = grp22.reset_index(level='customer_no')
grp23=pd.DataFrame(acc_train.groupby(["customer_no"])["mean_accs_weekly_paymentfrequency"].sum())
grp23.rename(columns={'mean_accs_weekly_paymentfrequency': 'total_accs_weekly_paymentfrequency'}, inplace=True)
grp23 = grp23.reset_index(level='customer_no')

acc_train['mean_rateofinterest'] = acc_train['rateofinterest'].astype(str)
acc_train['mean_rateofinterest'] = acc_train['mean_rateofinterest'].map(lambda x: x.lstrip('\'\"').rstrip('\'\"'))
acc_train["mean_rateofinterest"] = acc_train.mean_rateofinterest.astype(float)
acc_train['mean_rateofinterest'] = np.where(acc_train['mean_rateofinterest'].isnull(), 12.0 , acc_train['mean_rateofinterest'] )
grp24=pd.DataFrame(acc_train.groupby(["customer_no"])["mean_rateofinterest"].mean())
grp24 = grp24.reset_index(level='customer_no')

acc_train['mean_creditlimit'] = acc_train['creditlimit']
acc_train['mean_creditlimit'] = np.where(acc_train['mean_creditlimit'].isnull(), acc_train['high_credit_amt'], acc_train['mean_creditlimit'] )
acc_train['mean_creditlimit'] = np.where(acc_train['mean_creditlimit'].isnull(), acc_train['creditlimit'].mean(), acc_train['mean_creditlimit'] )
tmp1=pd.DataFrame(acc_train.groupby(["customer_no"])["mean_creditlimit"].mean())
tmp1 = tmp1.reset_index(level='customer_no')
tmp2=pd.DataFrame(acc_train.groupby(["customer_no"])["mean_creditlimit"].sum())
tmp2.rename(columns={'mean_creditlimit': 'total_creditlimit'}, inplace=True)
tmp2 = tmp2.reset_index(level='customer_no')

acc_train['mean_cashlimit'] = acc_train['cashlimit']
acc_train['mean_cashlimit'] = np.where(acc_train['mean_cashlimit'].isnull(), 0, acc_train['mean_cashlimit'] )
tmp3=pd.DataFrame(acc_train.groupby(["customer_no"])["mean_cashlimit"].mean())
tmp3 = tmp3.reset_index(level='customer_no')

acc_train['mean_cur_balance_amt'] = acc_train['cur_balance_amt']
tmp4=pd.DataFrame(acc_train.groupby(["customer_no"])["mean_cur_balance_amt"].mean())
tmp4 = tmp4.reset_index(level='customer_no')
tmp5=pd.DataFrame(acc_train.groupby(["customer_no"])["mean_cur_balance_amt"].sum())
tmp5.rename(columns={'mean_cur_balance_amt': 'total_cur_balance_amt'}, inplace=True)
tmp5 = tmp5.reset_index(level='customer_no')

tmp_cols = [tmp1,tmp2,tmp3,tmp4,tmp5]
tmp = reduce(lambda left,right: pd.merge(left,right,on='customer_no'), tmp_cols)
tmp['Ratio_currbalance_creditlimit'] = tmp['total_cur_balance_amt']/tmp['total_creditlimit']
tmp['utilisation_trend'] = (tmp['mean_cur_balance_amt']/(tmp['mean_creditlimit']+tmp['mean_cashlimit']))
tmp['utilisation_trend'] = tmp['Ratio_currbalance_creditlimit']/tmp['utilisation_trend']
tmp['utilisation_trend'] = np.where(tmp['utilisation_trend'].isnull(), tmp['Ratio_currbalance_creditlimit'],tmp['utilisation_trend'])
grp25 = pd.DataFrame(tmp[['customer_no', 'Ratio_currbalance_creditlimit', 'utilisation_trend']])

grp_cols = [grp1, grp2, grp3, grp4, grp5, grp6, grp7, grp8, grp9, grp10, grp11, grp12, grp13, grp14, grp15, grp16, grp17, grp18, grp19, grp20, grp21, grp22, grp23, grp24, grp25]
train_1 = reduce(lambda left,right: pd.merge(left,right,on='customer_no'), grp_cols)
train_1.head()

,customer_no,mean_accs_missing_last_paymt_dt,total_accs_missing_last_paymt_dt,mean_diff_lastpaymt_opened_dt,total_diff_lastpaymt_opened_dt,payment_history_mean_length,payment_history_total_length,ratio_closed_accs,total_closed_accs,most_common_account,...,total_accs_with_amt_past_due,mean_amt_past_due,total_amt_past_due,mean_accs_missing_paymentfrequency,total_accs_missing_paymentfrequency,mean_accs_weekly_paymentfrequency,total_accs_weekly_paymentfrequency,mean_rateofinterest,Ratio_currbalance_creditlimit,utilisation_trend
0,1,0.444444,8,2336.335711,42054.042806,12.555556,226,0.333333,6,10,...,2,141011.611111,2538209.0,0.611111,11,0.0,0,13.586333,0.562592,1.020046
1,2,0.000000,0,2035.000000,4070.000000,34.500000,69,1.000000,2,10,...,0,0.000000,0.0,1.000000,2,0.0,0,12.000000,0.027518,1.000001
2,3,0.000000,0,2454.000000,2454.000000,34.000000,34,1.000000,1,10,...,0,0.000000,0.0,1.000000,1,0.0,0,12.000000,0.147001,1.000000
3,4,0.176471,3,3035.941176,51611.000000,20.058824,341,0.294118,5,10,...,0,0.000000,0.0,0.705882,12,0.0,0,14.808235,0.141994,1.027775
4,5,0.714286,5,1743.000000,12201.000000,18.714286,131,0.285714,2,10,...,0,0.000000,0.0,0.714286,5,0.0,0,12.000000,0.000611,1.000000


### Enquiry Train Data Pre-Processing

In [1305]:
enq_train['mean_diff_open_enquiry_dt'] = (pd.to_datetime(enq_train["dt_opened"])-pd.to_datetime(enq_train["enquiry_dt"])).dt.days
enq_train['mean_diff_open_enquiry_dt'] = np.where(enq_train['mean_diff_open_enquiry_dt'].isnull(), 0, enq_train['mean_diff_open_enquiry_dt'] )
group1=pd.DataFrame(enq_train.groupby(["customer_no"])["mean_diff_open_enquiry_dt"].mean())
group1 = group1.reset_index(level='customer_no')
group2 = pd.DataFrame(enq_train.groupby(["customer_no"])["mean_diff_open_enquiry_dt"].sum())
group2.rename(columns={'mean_diff_open_enquiry_dt': 'total_diff_open_enquiry_dt'}, inplace=True)
group2 = group2.reset_index(level='customer_no')

enq_train['diff_upload_enquiry_dt'] = (pd.to_datetime(enq_train["upload_dt"])-pd.to_datetime(enq_train["enquiry_dt"])).dt.days
enq_train['count_enquiry_recency_90'] = np.where(enq_train['diff_upload_enquiry_dt'] <= 90, 1, 0 )
enq_train['count_enquiry_recency_90'] = np.where(enq_train['count_enquiry_recency_90'].isnull(), 0, enq_train['count_enquiry_recency_90'])
group3=pd.DataFrame(enq_train.groupby(["customer_no"])["count_enquiry_recency_90"].sum())
group3 = group3.reset_index(level='customer_no')
group4 = pd.DataFrame(enq_train.groupby(["customer_no"])["count_enquiry_recency_90"].mean())
group4.rename(columns={'count_enquiry_recency_90': 'mean_enquiry_recency_90'}, inplace=True)
group4 = group4.reset_index(level='customer_no')

enq_train['count_enquiry_recency_365'] = np.where(enq_train['diff_upload_enquiry_dt'] <= 365, 1, 0 )
enq_train['count_enquiry_recency_365'] = np.where(enq_train['count_enquiry_recency_365'].isnull(), 0, enq_train['count_enquiry_recency_365'])
group5 = pd.DataFrame(enq_train.groupby(["customer_no"])["count_enquiry_recency_365"].sum())
group5 = group5.reset_index(level='customer_no')
group6 = pd.DataFrame(enq_train.groupby(["customer_no"])["count_enquiry_recency_365"].mean())
group6.rename(columns={'count_enquiry_recency_365': 'mean_enquiry_recency_365'}, inplace=True)
group6 = group6.reset_index(level='customer_no')

enq_train['max_freq_enquiry'] = enq_train['enq_purpose']
#null values are imputed with 999 value
enq_train['max_freq_enquiry'] = np.where(enq_train['max_freq_enquiry'].isnull(),999,enq_train['max_freq_enquiry'])
group7 = pd.DataFrame(enq_train.groupby(['customer_no'])['max_freq_enquiry'].apply(lambda x: x.mode()))
group7 = group7.xs(0, level=1)
group7 = group7.reset_index(level='customer_no')

enq_train['no_enquiry'] = np.where(enq_train['enquiry_dt'].isnull(),1,0)
group8 = pd.DataFrame(enq_train.groupby(["customer_no"])["no_enquiry"].mean())
group8 = group8.reset_index(level='customer_no')

enq_train['perc_unsecured_others'] = np.where(enq_train['enq_purpose'].isin(sec_ln),1,0)
group9=pd.DataFrame(enq_train.groupby(["customer_no"])["perc_unsecured_others"].mean())
group9 = group9.reset_index(level='customer_no')

group_cols = [group1, group2, group3, group4, group5, group6, group7, group8, group9]
train_2 = reduce(lambda left,right: pd.merge(left,right,on='customer_no'), group_cols)
train_2.head()

,customer_no,mean_diff_open_enquiry_dt,total_diff_open_enquiry_dt,count_enquiry_recency_90,mean_enquiry_recency_90,count_enquiry_recency_365,mean_enquiry_recency_365,max_freq_enquiry,no_enquiry,perc_unsecured_others
0,1,1443.500000,25983.0,0,0.000000,1,0.055556,10.0,0,0.055556
1,2,778.940299,52189.0,2,0.029851,19,0.283582,51.0,0,0.552239
2,3,438.000000,438.0,0,0.000000,0,0.000000,1.0,0,1.000000
3,4,1903.303030,62809.0,0,0.000000,0,0.000000,5.0,0,0.515152
4,5,1497.500000,2995.0,0,0.000000,0,0.000000,10.0,0,0.000000


### Demographic Train Data Pre-Processing

In [1356]:
# features correlated, hard to interpret and which don't provide any information are dropped.
col_un=["feature_5","feature_6","feature_15","feature_16","feature_17","feature_18","feature_20","feature_22","feature_24", 
        'feature_36', 'feature_37', 'feature_46',"feature_51", "feature_28", "feature_38","feature_45","feature_47",
        "feature_49","feature_57","feature_61","feature_73","feature_74","feature_77", "feature_8", 
        "feature_54", "feature_2","feature_33","feature_34","feature_35","feature_70","feature_63",
        "feature_1", "feature_43", "feature_48", "feature_12", 'feature_32', 'feature_27' ]
data_train = data_train.drop(col_un,axis =1)
#  "feature_12", "feature_27", 'feature_32'
data_train['feature_9'] = np.where(data_train['feature_9'].isnull(), 0, 1)
data_train['feature_10'] = np.where(data_train['feature_10'].isnull(), 0, 1)
data_train['feature_13'] = np.where(data_train['feature_13'].isnull(), 0, 1)
data_train['feature_14'] = np.where(data_train['feature_14'].isnull(), 0, 1)
# data_train['feature_36'] = np.where(data_train['feature_36'].isnull(), 'Others', data_train['feature_36'])
# data_train['feature_37'] = np.where(data_train['feature_37'].isnull(), 'Others', data_train['feature_37'])
# data_train['feature_48'] = np.where(data_train['feature_48'].isnull(), 'Others', data_train['feature_48'])
# data_train['feature_51'] = np.where(data_train['feature_51'].isnull(), 'Others', data_train['feature_51'])
# data_train['feature_27'] = np.where(data_train['feature_27'].isnull(), 'Others', data_train['feature_27'])
# data_train['feature_27'] = np.where(data_train['feature_27'] == 'Others', 0, 1)
# data_train['feature_46'] = np.where(data_train['feature_46'].isnull(), 'None', data_train['feature_46'])
data_train['feature_53'] = np.where(data_train['feature_53'].isnull(), 0, 1)
data_train['missing_feature_3'] = np.where(data_train['feature_3'].isnull(), 1, 0)
data_train['feature_30'] = (2015.0 - data_train['feature_30'])
data_train['feature_39'] = np.where(data_train['feature_39']==0.0, 2015.0, data_train['feature_39'])
data_train['feature_39'] = (2015.0 - data_train['feature_39'])
data_train['feature_75'] = np.where(data_train['feature_75'] == '0000-00-00', 0, 1)
data_train['feature_3'] = np.where(data_train['feature_53'].isnull(), data_train['feature_53'].mean(), data_train['feature_53'])
data_train['feature_21'] = ((pd.to_datetime(data_train["dt_opened"])-pd.to_datetime(data_train["feature_21"]))/np.timedelta64(1, 'M'))

data_train = data_train.drop(['dt_opened', 'entry_time' ],axis =1)
data_train = data_train.fillna(method='ffill').fillna(method = 'ffill', axis=0).fillna(method = 'bfill', axis=0)
data_train['feature_72'] = np.where(data_train['feature_72'] == 'O', 0, 1)
train_cols = [train_1, train_2, data_train]
train = reduce(lambda left,right: pd.merge(left,right,on='customer_no'), train_cols)
train.head()

,customer_no,mean_accs_missing_last_paymt_dt,total_accs_missing_last_paymt_dt,mean_diff_lastpaymt_opened_dt,total_diff_lastpaymt_opened_dt,payment_history_mean_length,payment_history_total_length,ratio_closed_accs,total_closed_accs,most_common_account,...,feature_68,feature_69,feature_71,feature_72,feature_75,feature_76,feature_78,feature_79,Bad_label,missing_feature_3
0,1,0.444444,8,2336.335711,42054.042806,12.555556,226,0.333333,6,10,...,2.0,90000.0,21.0,1,0,0.0,1.0,N,0,0
1,2,0.000000,0,2035.000000,4070.000000,34.500000,69,1.000000,2,10,...,2.0,1.0,17.0,1,0,0.0,1.0,N,0,0
2,3,0.000000,0,2454.000000,2454.000000,34.000000,34,1.000000,1,10,...,2.0,1.0,17.0,1,0,0.0,1.0,N,0,0
3,4,0.176471,3,3035.941176,51611.000000,20.058824,341,0.294118,5,10,...,2.0,100000.0,21.0,1,1,1.0,1.0,N,0,0
4,5,0.714286,5,1743.000000,12201.000000,18.714286,131,0.285714,2,10,...,2.0,300000.0,13.0,1,0,0.0,1.0,N,0,1


### Account Test Data Pre-Processing

In [1307]:
acc_test['mean_accs_missing_last_paymt_dt'] = np.where(acc_test['last_paymt_dt'].isnull(),1,0)
grp1=pd.DataFrame(acc_test.groupby(["customer_no"])["mean_accs_missing_last_paymt_dt"].mean())
grp1 = grp1.reset_index(level='customer_no')
grp2=pd.DataFrame(acc_test.groupby(["customer_no"])["mean_accs_missing_last_paymt_dt"].sum())
grp2.rename(columns={'mean_accs_missing_last_paymt_dt': 'total_accs_missing_last_paymt_dt'}, inplace=True)
grp2 = grp2.reset_index(level='customer_no')

# fill NA values in last payment date with reporting date values.
# create feature null or not last payment date
acc_test['last_paymt_dt'] = np.where(acc_test['last_paymt_dt'].isnull(), acc_test['reporting_dt'], acc_test['last_paymt_dt'] )
acc_test['mean_diff_lastpaymt_opened_dt'] = (pd.to_datetime(acc_test["last_paymt_dt"])-pd.to_datetime(acc_test["opened_dt"])).dt.days
acc_test['mean_diff_lastpaymt_opened_dt'] = np.where(acc_test['mean_diff_lastpaymt_opened_dt'].isnull(), (pd.to_datetime(acc_test["last_paymt_dt"])-pd.to_datetime(acc_test["paymt_end_dt"])).dt.days - imp1, acc_test['mean_diff_lastpaymt_opened_dt'] )
grp3=pd.DataFrame(acc_test.groupby(["customer_no"])["mean_diff_lastpaymt_opened_dt"].mean())
grp3 = grp3.reset_index(level='customer_no')
grp4=pd.DataFrame(acc_test.groupby(["customer_no"])["mean_diff_lastpaymt_opened_dt"].sum())
grp4.rename(columns={'mean_diff_lastpaymt_opened_dt': 'total_diff_lastpaymt_opened_dt'}, inplace=True)
grp4 = grp4.reset_index(level='customer_no')

acc_test['payment_history_mean_length'] = ((pd.to_datetime(acc_test["paymt_str_dt"])-pd.to_datetime(acc_test["paymt_end_dt"]))/np.timedelta64(1, 'M'))
acc_test['payment_history_mean_length'] = np.where(acc_test['payment_history_mean_length'].isnull(), acc_test['payment_history_mean_length'].mean(), acc_test['payment_history_mean_length'] )
acc_test['payment_history_mean_length'] = acc_test['payment_history_mean_length'].astype(int)
grp5=pd.DataFrame(acc_test.groupby(["customer_no"])["payment_history_mean_length"].mean())
grp5 = grp5.reset_index(level='customer_no')
grp6=pd.DataFrame(acc_test.groupby(["customer_no"])["payment_history_mean_length"].sum())
grp6.rename(columns={'payment_history_mean_length': 'payment_history_total_length'}, inplace=True)
grp6 = grp6.reset_index(level='customer_no')

acc_test['ratio_closed_accs'] = np.where(acc_test['closed_dt'].isnull(),1,0)
grp7=pd.DataFrame(acc_test.groupby(["customer_no"])["ratio_closed_accs"].mean())
grp7 = grp7.reset_index(level='customer_no')
grp8=pd.DataFrame(acc_test.groupby(["customer_no"])["ratio_closed_accs"].sum())
grp8.rename(columns={'ratio_closed_accs': 'total_closed_accs'}, inplace=True)
grp8 = grp8.reset_index(level='customer_no')

acc_test['most_common_account'] = acc_test['acct_type']
grp9 = pd.DataFrame(acc_test.groupby(['customer_no'])['most_common_account'].apply(lambda x: x.mode()))
grp9 = grp9.xs(0, level=1)
grp9 = grp9.reset_index(level='customer_no')

acc_test['secured_accounts_ratio'] = np.where(acc_test['acct_type'].isin(sec_ln),1,0)
grp10=pd.DataFrame(acc_test.groupby(["customer_no"])["secured_accounts_ratio"].mean())
grp10 = grp10.reset_index(level='customer_no')
grp11=pd.DataFrame(acc_test.groupby(["customer_no"])["secured_accounts_ratio"].sum())
grp11.rename(columns={'secured_accounts_ratio': 'secured_accounts_total'}, inplace=True)
grp11 = grp11.reset_index(level='customer_no')

acc_test['creditcard_accounts_ratio'] = np.where(acc_test['acct_type'] == 10.0,1,0)
grp12=pd.DataFrame(acc_test.groupby(["customer_no"])["creditcard_accounts_ratio"].mean())
grp12 = grp12.reset_index(level='customer_no')
grp13=pd.DataFrame(acc_test.groupby(["customer_no"])["creditcard_accounts_ratio"].sum())
grp13.rename(columns={'creditcard_accounts_ratio': 'total_creditcard_accounts'}, inplace=True)
grp13 = grp13.reset_index(level='customer_no')

acc_test['mean_accs_individual_owner'] = np.where(acc_test['owner_indic'] == 1,1,0)
grp14=pd.DataFrame(acc_test.groupby(["customer_no"])["mean_accs_individual_owner"].mean())
grp14 = grp14.reset_index(level='customer_no')
grp15=pd.DataFrame(acc_test.groupby(["customer_no"])["mean_accs_individual_owner"].sum())
grp15.rename(columns={'mean_accs_individual_owner': 'total_accs_individual_owner'}, inplace=True)
grp15 = grp15.reset_index(level='customer_no')

acc_test['mean_accs_with_amt_past_due'] = np.where(acc_test['amt_past_due'].isnull(),0,1)
grp16=pd.DataFrame(acc_test.groupby(["customer_no"])["mean_accs_with_amt_past_due"].mean())
grp16 = grp16.reset_index(level='customer_no')
grp17=pd.DataFrame(acc_test.groupby(["customer_no"])["mean_accs_with_amt_past_due"].sum())
grp17.rename(columns={'mean_accs_with_amt_past_due': 'total_accs_with_amt_past_due'}, inplace=True)
grp17 = grp17.reset_index(level='customer_no')

acc_test['mean_amt_past_due'] = acc_test['amt_past_due']
acc_test['mean_amt_past_due'] = np.where(acc_test['mean_amt_past_due'].isnull(), 0, acc_test['mean_amt_past_due'] )
grp18=pd.DataFrame(acc_test.groupby(["customer_no"])["mean_amt_past_due"].mean())
grp18 = grp18.reset_index(level='customer_no')
grp19=pd.DataFrame(acc_test.groupby(["customer_no"])["mean_amt_past_due"].sum())
grp19.rename(columns={'mean_amt_past_due': 'total_amt_past_due'}, inplace=True)
grp19 = grp19.reset_index(level='customer_no')

acc_test['mean_accs_missing_paymentfrequency'] = np.where(acc_test['paymentfrequency'].isnull(),1,0)
grp20=pd.DataFrame(acc_test.groupby(["customer_no"])["mean_accs_missing_paymentfrequency"].mean())
grp20 = grp20.reset_index(level='customer_no')
grp21=pd.DataFrame(acc_test.groupby(["customer_no"])["mean_accs_missing_paymentfrequency"].sum())
grp21.rename(columns={'mean_accs_missing_paymentfrequency': 'total_accs_missing_paymentfrequency'}, inplace=True)
grp21 = grp21.reset_index(level='customer_no')

acc_test['mean_accs_weekly_paymentfrequency'] = np.where(acc_test['paymentfrequency'] == 1,1,0)
grp22=pd.DataFrame(acc_test.groupby(["customer_no"])["mean_accs_weekly_paymentfrequency"].mean())
grp22 = grp22.reset_index(level='customer_no')
grp23=pd.DataFrame(acc_test.groupby(["customer_no"])["mean_accs_weekly_paymentfrequency"].sum())
grp23.rename(columns={'mean_accs_weekly_paymentfrequency': 'total_accs_weekly_paymentfrequency'}, inplace=True)
grp23 = grp23.reset_index(level='customer_no')

acc_test['mean_rateofinterest'] = acc_test['rateofinterest'].astype(str)
acc_test['mean_rateofinterest'] = acc_test['mean_rateofinterest'].map(lambda x: x.lstrip('\'\"').rstrip('\'\"'))
acc_test["mean_rateofinterest"] = acc_test.mean_rateofinterest.astype(float)
acc_test['mean_rateofinterest'] = np.where(acc_test['mean_rateofinterest'].isnull(), 12.0 , acc_test['mean_rateofinterest'] )
grp24=pd.DataFrame(acc_test.groupby(["customer_no"])["mean_rateofinterest"].mean())
grp24 = grp24.reset_index(level='customer_no')

acc_test['mean_creditlimit'] = acc_test['creditlimit']
acc_test['mean_creditlimit'] = np.where(acc_test['mean_creditlimit'].isnull(), acc_test['high_credit_amt'], acc_test['mean_creditlimit'] )
acc_test['mean_creditlimit'] = np.where(acc_test['mean_creditlimit'].isnull(), acc_test['creditlimit'].mean(), acc_test['mean_creditlimit'] )
tmp1=pd.DataFrame(acc_test.groupby(["customer_no"])["mean_creditlimit"].mean())
tmp1 = tmp1.reset_index(level='customer_no')
tmp2=pd.DataFrame(acc_test.groupby(["customer_no"])["mean_creditlimit"].sum())
tmp2.rename(columns={'mean_creditlimit': 'total_creditlimit'}, inplace=True)
tmp2 = tmp2.reset_index(level='customer_no')

acc_test['mean_cashlimit'] = acc_test['cashlimit']
acc_test['mean_cashlimit'] = np.where(acc_test['mean_cashlimit'].isnull(), 0, acc_test['mean_cashlimit'] )
tmp3=pd.DataFrame(acc_test.groupby(["customer_no"])["mean_cashlimit"].mean())
tmp3 = tmp3.reset_index(level='customer_no')

acc_test['mean_cur_balance_amt'] = acc_test['cur_balance_amt']
tmp4=pd.DataFrame(acc_test.groupby(["customer_no"])["mean_cur_balance_amt"].mean())
tmp4 = tmp4.reset_index(level='customer_no')
tmp5=pd.DataFrame(acc_test.groupby(["customer_no"])["mean_cur_balance_amt"].sum())
tmp5.rename(columns={'mean_cur_balance_amt': 'total_cur_balance_amt'}, inplace=True)
tmp5 = tmp5.reset_index(level='customer_no')

tmp_cols = [tmp1,tmp2,tmp3,tmp4,tmp5]
tmp = reduce(lambda left,right: pd.merge(left,right,on='customer_no'), tmp_cols)
tmp['Ratio_currbalance_creditlimit'] = tmp['total_cur_balance_amt']/tmp['total_creditlimit']
tmp['utilisation_trend'] = (tmp['mean_cur_balance_amt']/(tmp['mean_creditlimit']+tmp['mean_cashlimit']))
tmp['utilisation_trend'] = tmp['Ratio_currbalance_creditlimit']/tmp['utilisation_trend']
tmp['utilisation_trend'] = np.where(tmp['utilisation_trend'].isnull(), tmp['Ratio_currbalance_creditlimit'],tmp['utilisation_trend'])
grp25 = pd.DataFrame(tmp[['customer_no', 'Ratio_currbalance_creditlimit', 'utilisation_trend']])

grp_cols = [grp1, grp2, grp3, grp4, grp5, grp6, grp7, grp8, grp9, grp10, grp11, grp12, grp13, grp14, grp15, grp16, grp17, grp18, grp19, grp20, grp21, grp22, grp23, grp24, grp25]
test_1 = reduce(lambda left,right: pd.merge(left,right,on='customer_no'), grp_cols)
test_1.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10240 entries, 0 to 10239
Data columns (total 27 columns):
customer_no                            10240 non-null int64
mean_accs_missing_last_paymt_dt        10240 non-null float64
total_accs_missing_last_paymt_dt       10240 non-null int32
mean_diff_lastpaymt_opened_dt          10240 non-null float64
total_diff_lastpaymt_opened_dt         10240 non-null float64
payment_history_mean_length            10240 non-null float64
payment_history_total_length           10240 non-null int32
ratio_closed_accs                      10240 non-null float64
total_closed_accs                      10240 non-null int32
most_common_account                    10240 non-null int64
secured_accounts_ratio                 10240 non-null float64
secured_accounts_total                 10240 non-null int32
creditcard_accounts_ratio              10240 non-null float64
total_creditcard_accounts              10240 non-null int32
mean_accs_individual_owner           

### Enquiry Test Data Pre-Processing

In [1308]:
enq_test['mean_diff_open_enquiry_dt'] = (pd.to_datetime(enq_test["dt_opened"])-pd.to_datetime(enq_test["enquiry_dt"])).dt.days
enq_test['mean_diff_open_enquiry_dt'] = np.where(enq_test['mean_diff_open_enquiry_dt'].isnull(), 0, enq_test['mean_diff_open_enquiry_dt'] )
group1=pd.DataFrame(enq_test.groupby(["customer_no"])["mean_diff_open_enquiry_dt"].mean())
group1 = group1.reset_index(level='customer_no')
group2 = pd.DataFrame(enq_test.groupby(["customer_no"])["mean_diff_open_enquiry_dt"].sum())
group2.rename(columns={'mean_diff_open_enquiry_dt': 'total_diff_open_enquiry_dt'}, inplace=True)
group2 = group2.reset_index(level='customer_no')

enq_test['diff_upload_enquiry_dt'] = (pd.to_datetime(enq_test["upload_dt"])-pd.to_datetime(enq_test["enquiry_dt"])).dt.days
enq_test['count_enquiry_recency_90'] = np.where(enq_test['diff_upload_enquiry_dt'] <= 90, 1, 0 )
enq_test['count_enquiry_recency_90'] = np.where(enq_test['count_enquiry_recency_90'].isnull(), 0, enq_test['count_enquiry_recency_90'])
group3=pd.DataFrame(enq_test.groupby(["customer_no"])["count_enquiry_recency_90"].sum())
group3 = group3.reset_index(level='customer_no')
group4 = pd.DataFrame(enq_test.groupby(["customer_no"])["count_enquiry_recency_90"].mean())
group4.rename(columns={'count_enquiry_recency_90': 'mean_enquiry_recency_90'}, inplace=True)
group4 = group4.reset_index(level='customer_no')

enq_test['count_enquiry_recency_365'] = np.where(enq_test['diff_upload_enquiry_dt'] <= 365, 1, 0 )
enq_test['count_enquiry_recency_365'] = np.where(enq_test['count_enquiry_recency_365'].isnull(), 0, enq_test['count_enquiry_recency_365'])
group5 = pd.DataFrame(enq_test.groupby(["customer_no"])["count_enquiry_recency_365"].sum())
group5 = group5.reset_index(level='customer_no')
group6 = pd.DataFrame(enq_test.groupby(["customer_no"])["count_enquiry_recency_365"].mean())
group6.rename(columns={'count_enquiry_recency_365': 'mean_enquiry_recency_365'}, inplace=True)
group6 = group6.reset_index(level='customer_no')

enq_test['max_freq_enquiry'] = enq_test['enq_purpose']
#null values are imputed with 999 value
enq_test['max_freq_enquiry'] = np.where(enq_test['max_freq_enquiry'].isnull(),999,enq_test['max_freq_enquiry'])
group7 = pd.DataFrame(enq_test.groupby(['customer_no'])['max_freq_enquiry'].apply(lambda x: x.mode()))
group7 = group7.xs(0, level=1)
group7 = group7.reset_index(level='customer_no')

enq_test['no_enquiry'] = np.where(enq_test['enquiry_dt'].isnull(),1,0)
group8 = pd.DataFrame(enq_test.groupby(["customer_no"])["no_enquiry"].mean())
group8 = group8.reset_index(level='customer_no')

enq_test['perc_unsecured_others'] = np.where(enq_test['enq_purpose'].isin(sec_ln),1,0)
group9=pd.DataFrame(enq_test.groupby(["customer_no"])["perc_unsecured_others"].mean())
group9 = group9.reset_index(level='customer_no')

group_cols = [group1, group2, group3, group4, group5, group6, group7, group8, group9]
test_2 = reduce(lambda left,right: pd.merge(left,right,on='customer_no'), group_cols)
test_2.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10240 entries, 0 to 10239
Data columns (total 10 columns):
customer_no                   10240 non-null int64
mean_diff_open_enquiry_dt     10240 non-null float64
total_diff_open_enquiry_dt    10240 non-null float64
count_enquiry_recency_90      10240 non-null int32
mean_enquiry_recency_90       10240 non-null float64
count_enquiry_recency_365     10240 non-null int32
mean_enquiry_recency_365      10240 non-null float64
max_freq_enquiry              10240 non-null float64
no_enquiry                    10240 non-null int32
perc_unsecured_others         10240 non-null float64
dtypes: float64(6), int32(3), int64(1)
memory usage: 760.0 KB


### Demographic Test Data Pre-Processing

In [1357]:
data_test = data_test.drop(col_un,axis =1)

data_test['feature_9'] = np.where(data_test['feature_9'].isnull(), 0, 1)
data_test['feature_10'] = np.where(data_test['feature_10'].isnull(), 0, 1)
data_test['feature_13'] = np.where(data_test['feature_13'].isnull(), 0, 1)
data_test['feature_14'] = np.where(data_test['feature_14'].isnull(), 0, 1)
# data_test['feature_36'] = np.where(data_test['feature_36'].isnull(), 'Others', data_test['feature_36'])
# data_test['feature_37'] = np.where(data_test['feature_37'].isnull(), 'Others', data_test['feature_37'])
# data_test['feature_48'] = np.where(data_test['feature_48'].isnull(), 'Others', data_test['feature_48'])
# data_test['feature_51'] = np.where(data_test['feature_51'].isnull(), 'Others', data_test['feature_51'])
# data_test['feature_27'] = np.where(data_test['feature_27'].isnull(), 'Others', data_test['feature_27'])
# data_test['feature_27'] = np.where(data_test['feature_27'] == 'Others', 0, 1)
# data_test['feature_46'] = np.where(data_test['feature_46'].isnull(), 'None', data_test['feature_46'])
data_test['feature_53'] = np.where(data_test['feature_53'].isnull(), 0, 1)
data_test['missing_feature_3'] = np.where(data_test['feature_3'].isnull(), 1, 0)
data_test['feature_30'] = (2015.0 - data_test['feature_30'])
data_test['feature_39'] = np.where(data_test['feature_39']==0.0, 2015.0, data_test['feature_39'])
data_test['feature_39'] = (2015.0 - data_test['feature_39'])
data_test['feature_75'] = np.where(data_test['feature_75'] == '0000-00-00', 0, 1)
data_test['feature_3'] = np.where(data_test['feature_53'].isnull(), data_test['feature_53'].mean(), data_test['feature_53'])
data_test['feature_21'] = ((pd.to_datetime(data_test["dt_opened"])-pd.to_datetime(data_test["feature_21"]))/np.timedelta64(1, 'M'))

data_test = data_test.drop(['dt_opened', 'entry_time' ],axis =1)
data_test = data_test.fillna(method='ffill').fillna(method = 'ffill', axis=0).fillna(method = 'bfill', axis=0)
data_test['feature_72'] = np.where(data_test['feature_72'] == 'O', 0, 1)
test_cols = [test_1, test_2, data_test]
test = reduce(lambda left,right: pd.merge(left,right,on='customer_no'), test_cols)
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10240 entries, 0 to 10239
Data columns (total 80 columns):
customer_no                            10240 non-null int64
mean_accs_missing_last_paymt_dt        10240 non-null float64
total_accs_missing_last_paymt_dt       10240 non-null int32
mean_diff_lastpaymt_opened_dt          10240 non-null float64
total_diff_lastpaymt_opened_dt         10240 non-null float64
payment_history_mean_length            10240 non-null float64
payment_history_total_length           10240 non-null int32
ratio_closed_accs                      10240 non-null float64
total_closed_accs                      10240 non-null int32
most_common_account                    10240 non-null int64
secured_accounts_ratio                 10240 non-null float64
secured_accounts_total                 10240 non-null int32
creditcard_accounts_ratio              10240 non-null float64
total_creditcard_accounts              10240 non-null int32
mean_accs_individual_owner           

##### One hot encoding categorical variables

In [1358]:
one_hot_encoded_training_predictors = pd.get_dummies(train, drop_first=True)        
one_hot_encoded_test_predictors = pd.get_dummies(test, drop_first=True)
final_train, final_test = one_hot_encoded_training_predictors.align(one_hot_encoded_test_predictors,
                                                                    join='inner', 
                                                                    axis=1)
final_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10240 entries, 0 to 10239
Data columns (total 80 columns):
customer_no                            10240 non-null int64
mean_accs_missing_last_paymt_dt        10240 non-null float64
total_accs_missing_last_paymt_dt       10240 non-null int32
mean_diff_lastpaymt_opened_dt          10240 non-null float64
total_diff_lastpaymt_opened_dt         10240 non-null float64
payment_history_mean_length            10240 non-null float64
payment_history_total_length           10240 non-null int32
ratio_closed_accs                      10240 non-null float64
total_closed_accs                      10240 non-null int32
most_common_account                    10240 non-null int64
secured_accounts_ratio                 10240 non-null float64
secured_accounts_total                 10240 non-null int32
creditcard_accounts_ratio              10240 non-null float64
total_creditcard_accounts              10240 non-null int32
mean_accs_individual_owner           

In [1311]:
# f1 = data_test['feature_32'].unique()
# f1

In [1390]:
# Create correlation matrix
corr_matrix = final_train.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.90)]
# # Drop features 
final_train = final_train.drop(to_drop, axis=1)
final_test = final_test.drop(to_drop, axis=1)
print(to_drop)
final_train.info()

['total_accs_individual_owner', 'total_accs_missing_paymentfrequency', 'feature_55']
<class 'pandas.core.frame.DataFrame'>
Int64Index: 23896 entries, 0 to 23895
Data columns (total 71 columns):
customer_no                           23896 non-null int64
mean_accs_missing_last_paymt_dt       23896 non-null float64
total_accs_missing_last_paymt_dt      23896 non-null int32
mean_diff_lastpaymt_opened_dt         23896 non-null float64
total_diff_lastpaymt_opened_dt        23896 non-null float64
payment_history_mean_length           23896 non-null float64
payment_history_total_length          23896 non-null int32
ratio_closed_accs                     23896 non-null float64
total_closed_accs                     23896 non-null int32
most_common_account                   23896 non-null int64
secured_accounts_ratio                23896 non-null float64
secured_accounts_total                23896 non-null int32
creditcard_accounts_ratio             23896 non-null float64
total_creditcard_accounts

##### Separating labels and features of both Test and Train Data

In [1391]:
# cols = ['total_diff_lastpaymt_opened_dt', 'utilisation_trend', 'count_enquiry_recency_365',
#         'Ratio_currbalance_creditlimit', 'mean_diff_lastpaymt_opened_dt', 'mean_diff_open_enquiry_dt',
#         'payment_history_mean_length', 'max_freq_enquiry', 'count_enquiry_recency_90', 'perc_unsecured_others']
# col2 = ['customer_no', 'mean_accs_missing_last_paymt_dt',
#        'total_accs_missing_last_paymt_dt',
#        'total_diff_lastpaymt_opened_dt',
#        'payment_history_total_length', 'ratio_closed_accs',
#        'total_closed_accs', 'most_common_account', 'secured_accounts_ratio',
#        'secured_accounts_total', 'creditcard_accounts_ratio',
#        'total_creditcard_accounts', 'mean_accs_individual_owner',
#        'total_accs_individual_owner', 'mean_accs_with_amt_past_due',
#        'total_accs_with_amt_past_due', 'mean_amt_past_due',
#        'total_amt_past_due', 'mean_accs_missing_paymentfrequency',
#        'total_accs_missing_paymentfrequency',
#        'mean_accs_weekly_paymentfrequency',
#        'total_accs_weekly_paymentfrequency', 'mean_rateofinterest', 'total_diff_open_enquiry_dt', 'mean_enquiry_recency_90',
#         'mean_enquiry_recency_365','no_enquiry']

Y_train = final_train['Bad_label']
X_train = final_train.drop(['Bad_label', 'customer_no'], axis=1)
# final_train.drop(['Bad_label', 'customer_no'], axis=1)
# train_2.drop(['customer_no'], axis=1)
Y_test = final_test['Bad_label']
X_test = final_test.drop(['Bad_label', 'customer_no'], axis=1)
# final_test.drop(['Bad_label', 'customer_no'], axis=1)
# test_2.drop(['customer_no'], axis=1)

In [1445]:
# Define the model. Set random_state to 1
rf_model = RandomForestClassifier(n_estimators=500, random_state =1, criterion = 'entropy', class_weight ='balanced')
from sklearn.feature_selection import SelectFromModel
# LogisticRegression()
# RandomForestClassifier(random_state =1)
# fit your model
rf_model.fit(X_train, Y_train)
preds = rf_model.predict(X_test)
print("accuracy = ",accuracy_score(Y_test, preds))
print("classification report", classification_report(Y_test, preds))
print("confusion matrix\n", confusion_matrix(Y_test, preds))
fpr, tpr, thresholds = metrics.roc_curve(Y_test, preds)
auc = metrics.auc(fpr, tpr)
print('Gini = ', 2*auc-1)

accuracy =  0.9548828125
classification report              precision    recall  f1-score   support

          0       0.95      1.00      0.98      9778
          1       0.00      0.00      0.00       462

avg / total       0.91      0.95      0.93     10240

confusion matrix
 [[9778    0]
 [ 462    0]]
Gini =  0.0


(23896, 63)

### List of features selected with gain

Used random forests to find the information gains for feature selection with a selection criteria parameter given to the model as 'entropy'

In [1449]:

importances = rf_model.feature_importances_
indices = np.argsort(importances)[::-1]
# Rearrange feature names so they match the sorted feature importances
names = [X_train.columns[i] for i in indices]

res = dict(zip(names,
               importances[indices]
               ))
for name, importance in res.items():
    print(name, "=", importance)

feature_7 = 0.04525680215857141
Ratio_currbalance_creditlimit = 0.03984022378039549
feature_21 = 0.03807384241759036
mean_diff_open_enquiry_dt = 0.03724240589621233
mean_diff_lastpaymt_opened_dt = 0.036211037140965685
total_diff_open_enquiry_dt = 0.03606710577583873
total_diff_lastpaymt_opened_dt = 0.03500165655685021
payment_history_mean_length = 0.033698741973028315
utilisation_trend = 0.033242085751271036
payment_history_total_length = 0.032495238796276436
feature_66 = 0.0319686197840028
mean_enquiry_recency_365 = 0.03148827266588891
feature_29 = 0.03089662323078329
feature_30 = 0.0288611691016141
mean_enquiry_recency_90 = 0.024717238495303686
mean_accs_missing_paymentfrequency = 0.023791665383633257
count_enquiry_recency_365 = 0.02352070073117609
perc_unsecured_others = 0.023195637537900864
feature_69 = 0.02288036663155766
mean_rateofinterest = 0.022874242249756965
creditcard_accounts_ratio = 0.022841422976496637
feature_52 = 0.02281295300855775
feature_39 = 0.019870853328220115
ra

##### As the Data is very Imbalanced needs to try both oversampling and undersampling techniques

### Under-sampling majority class using  a Balanced Classifier 

In [1444]:
# log_reg_params = {"penalty": ['l1', 'l2'], 'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}
bbc = BalancedBaggingClassifier(base_estimator=XGBClassifier(random_state =1, learning_rate=0.3, n_estimators= 300, max_depth=1),
                                replacement=False,random_state=0, bootstrap_features=False)
bbc.fit(pd.DataFrame(X_train), Y_train)
preds = bbc.predict(X_test)
print("accuracy = ",accuracy_score(Y_test, preds))
print("classification report\n", classification_report(Y_test, preds))
print("confusion matrix\n", confusion_matrix(Y_test, preds))
# XGBClassifier(random_state =1, learning_rate=0.25, n_estimators= 300, max_depth=1)
# GradientBoostingClassifier(random_state =1, learning_rate=0.12, n_estimators=100, max_depth=1, min_samples_leaf=50)
# RandomForestClassifier(n_estimators=500, random_state =1, min_samples_split= 75)
# base_estimator=RandomForestClassifier(n_estimators=100,

accuracy =  0.65009765625
classification report
              precision    recall  f1-score   support

          0       0.97      0.65      0.78      9778
          1       0.08      0.62      0.14       462

avg / total       0.93      0.65      0.75     10240

confusion matrix
 [[6372 3406]
 [ 177  285]]


In [1428]:
fpr, tpr, thresholds = metrics.roc_curve(Y_test, preds)
auc = metrics.auc(fpr, tpr)
print('Gini = ', 2*auc-1)

Gini =  0.2685501244511266


### Oversampling minority class using SMOTE technique  

In [1443]:
sm = SMOTE(random_state=42)
pipeline = Pipeline([('smote', sm), ('Lr', LogisticRegression(penalty = 'l1', C = 100, random_state=42) )])
# LogisticRegression(penalty = 'l1', C = 10, random_state=42)
pipeline.fit(X_train, Y_train)
preds = pipeline.predict(X_test)
print("accuracy = ",accuracy_score(Y_test, preds))
print("classification report\n", classification_report(Y_test, preds))
# print("confusion matrix\n", confusion_matrix(Y_test, preds))


accuracy =  0.63349609375
classification report
              precision    recall  f1-score   support

          0       0.97      0.63      0.77      9778
          1       0.08      0.65      0.14       462

avg / total       0.93      0.63      0.74     10240

confusion matrix
 [[6188 3590]
 [ 163  299]]


In [1433]:
fpr, tpr, thresholds = metrics.roc_curve(Y_test, preds)
auc = metrics.auc(fpr, tpr)
print('Gini = ', 2*auc-1)

Gini =  0.2800354006122059


The **Logistic Regression** Model built with 69 features using **SMOTE** sampling technique to counter the imbalanced classes has achieved a **GINI** of **28%**